In [1]:
# https://www.kaggle.com/c/house-prices-advanced-regression-techniques/overview/evaluation
from kaggle import api
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from fastai.tabular.all import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
#from dtreeviz.trees import *
from IPython.display import Image, display_svg, SVG

pd.options.display.max_rows = 20
pd.options.display.max_columns = 8

In [2]:
path = URLs.path('housedata')
path

Path('/home/gsimpson/.fastai/archive/housedata')

In [3]:
dataset_name = 'house-prices-advanced-regression-techniques'
dataset_zip = dataset_name + '.zip'
if not path.exists():
    path.mkdir(parents=True, exist_ok=True)
    api.competition_download_cli(dataset_name, path=path)
    file_extract(path/dataset_zip)

path.ls(file_type='text')

(#4) [Path('/home/gsimpson/.fastai/archive/housedata/sample_submission.csv'),Path('/home/gsimpson/.fastai/archive/housedata/test.csv'),Path('/home/gsimpson/.fastai/archive/housedata/train.csv'),Path('/home/gsimpson/.fastai/archive/housedata/data_description.txt')]

In [12]:
df = pd.read_csv(path/'train.csv', low_memory=False)
test = pd.read_csv(path/'test.csv', low_memory=False)
# df = pd.concat([train, test])
# df.columns

In [13]:
# for col in df.columns:
#     print(col)
#     if len(df[col].unique()) < 10: print(df[col].unique())
#     else: print('Too many values')

In [14]:
import read_descriptions
description_file = path/'data_description.txt'
categorical_dictionary = read_descriptions.parse_file(read_descriptions.read_file(description_file))
print(categorical_dictionary.keys())

dict_keys(['MSSubClass', 'MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition'])


In [15]:
df['SalePrice'].dropna(inplace=True)

for category, values in categorical_dictionary.items():
    df[category] = df[category].astype('category')
    df[category].cat.set_categories(tuple(values), ordered=True, inplace=True)
    #catcol = pd.Series(df[category], dtype='category')

numerical_categories = list(df) - categorical_dictionary.keys()
for category in numerical_categories:
    df[category].fillna(0, inplace=True)
    #print(np.all(np.isfinite(df[category])), category, min(df[category]), max(df[category]))

In [16]:
dep_var = 'SalePrice' # Dependent Variable
df[dep_var] = np.log(df[dep_var])

In [17]:
procs = [Categorify, FillMissing]

In [19]:
splits = RandomSplitter(valid_pct=0.1)(range_of(df))

In [26]:
cont, cat = cont_cat_split(df, 20, dep_var=dep_var)
to = TabularPandas(df, procs, cat, cont, y_names=dep_var, splits=splits)
print(len(to.train), len(to.valid))
xs, y = to.train.xs, to.train.y
valid_xs, valid_y = to.valid.xs, to.valid.y

1314 146


In [27]:
def rf(xs, y, n_estimators=40, max_samples=2_000,
       max_features=0.5, min_samples_leaf=5, **kwargs):
    return RandomForestRegressor(n_jobs=-1, n_estimators=n_estimators,
        max_samples=max_samples, max_features=max_features,
        min_samples_leaf=min_samples_leaf, oob_score=True).fit(xs, y)

In [28]:
def r_mse(pred,y): return round(math.sqrt(((pred-y)**2).mean()), 6)
def m_rmse(m, xs, y): return r_mse(m.predict(xs), y)

In [29]:
m = rf(xs, y, max_samples=len(xs))
m_rmse(m, xs, y), m_rmse(m, valid_xs, valid_y)

(0.101689, 0.152527)